# KnowledgeGraph Demo

A flat, SQLite-native knowledge graph with YAML frontmatter and `[[wikilinks]]`.

Zero external dependencies — just `sqlite3` from stdlib.

Nodes are identified by name (auto-slugified). `type` is the grouping mechanism — no directories.

In [ ]:
from loopy import KnowledgeGraph

## 1. Basics

Create a graph, register types, add nodes.

In [ ]:
kg = KnowledgeGraph()  # :memory: by default

kg.add_type("concept")
kg.add_type("person")
kg.touch("readme", "Welcome to the knowledge graph.")

kg.ls()  # lists registered types

In [ ]:
kg.cat("readme")

## 2. Frontmatter + typed nodes

Write content with YAML frontmatter. The `type` field groups the node and auto-creates a SQL table.

In [ ]:
kg.write("spreading-activation", """---
type: concept
description: How activation propagates through a network
tags: [graph, cognition, search]
---
Spreading activation is a method for searching associative networks.
It follows [[agent-traversal]] paths and uses [[semantic-similarity]].""")

kg.write("agent-traversal", """---
type: concept
description: How agents navigate graph structures
tags: [graph, agents]
---
Traversal strategies let agents explore graphs efficiently.
Often combined with [[spreading-activation]] for discovery.""")

kg.write("semantic-similarity", """---
type: concept
description: Measuring meaning overlap between items
tags: [nlp, embeddings]
---
Semantic similarity quantifies how close two meanings are.""")

kg.write("turing", """---
type: person
role: researcher
born: 1912
---
Alan Turing pioneered computation theory and [[agent-traversal]].""")

print("Done writing 4 nodes.")

### Inspect frontmatter and body separately

In [ ]:
kg.frontmatter("spreading-activation")

In [ ]:
kg.body("spreading-activation")

In [ ]:
# cat() reconstructs the full document (frontmatter + body)
print(kg.cat("spreading-activation"))

## 3. Wikilinks + graph traversal

`[[wikilinks]]` are extracted on write and stored as edges in a `_links` table.

In [ ]:
kg.wikilinks("spreading-activation")

In [ ]:
# Backlinks: who points TO agent-traversal?
kg.backlinks("agent-traversal")

In [ ]:
# Full adjacency list
kg.graph()

### Fuzzy wikilink resolution

Resolve human-readable names to node names using `slugify` matching.

In [ ]:
kg.resolve_wikilink("Agent Traversal")  # fuzzy: spaces -> hyphens, case-insensitive

In [ ]:
kg.resolve_wikilink("nonexistent")  # returns None

## 4. Types and ls()

`ls()` lists registered types. `ls("concept")` lists nodes of that type. `ls("*")` lists all nodes.

In [ ]:
kg.ls()  # types

In [ ]:
kg.ls("concept")  # nodes of type

In [ ]:
kg.ls("*")  # all nodes

## 5. Dynamic per-type SQL tables

Each `type` in frontmatter gets its own SQL table with columns for every key seen.

In [ ]:
kg.schema()

In [ ]:
kg.find_by_type("concept")

In [ ]:
# Query the auto-created 'concept' table directly
kg.query("SELECT name, description, tags FROM concept ORDER BY name")

In [ ]:
# Query the 'person' table
kg.query("SELECT name, role, born FROM person")

## 6. Tags

In [ ]:
# Tags for a specific node
kg.tags("spreading-activation")

In [ ]:
# Tag -> nodes mapping
kg.tags()

## 7. Tree, find, grep

In [ ]:
print(kg.tree())

In [ ]:
kg.find(name="spread.*")  # regex name match

In [ ]:
kg.grep("graph", content=True)  # search content

In [ ]:
kg.grep("agent", lines=True)  # line-level matches

## 8. mv, cp, rm

In [ ]:
kg.cp("semantic-similarity", "sem-sim-backup")
kg.ls("concept")

In [ ]:
# Remove the backup — type table and links are cleaned up automatically
kg.rm("sem-sim-backup")
kg.ls("concept")

## 9. Symlinks

In [ ]:
kg.ln("agent-traversal", "traversal-ref")

# The symlink shows up in backlinks
kg.backlinks("agent-traversal")

## 10. Shell integration

Graph commands plug into the Loopy shell. The `run()` function works via duck typing.

In [ ]:
from loopy.shell import run, COMMANDS
from loopy.graph import register_graph_commands

register_graph_commands(COMMANDS)

In [ ]:
print(run("meta spreading-activation", kg))

In [ ]:
print(run("links spreading-activation", kg))

In [ ]:
print(run("backlinks agent-traversal", kg))

In [ ]:
print(run("schema", kg))

In [ ]:
# Piping works too
print(run("query SELECT name, description FROM concept | grep -i graph", kg))

## 11. Direct SQL

`query()` gives you full SQL access to the underlying SQLite database.

In [ ]:
# How many nodes of each type?
kg.query("SELECT type, COUNT(*) FROM nodes WHERE type IS NOT NULL GROUP BY type")

In [ ]:
# All edges in the link graph
kg.query("SELECT source_name, target_name, target_resolved FROM _links ORDER BY source_name")

In [ ]:
# Join: concepts that are linked to by people
kg.query("""
    SELECT l.source_name AS person, l.target_resolved AS concept
    FROM _links l
    JOIN nodes n ON n.name = l.source_name
    WHERE n.type = 'person'
""")

## 12. Persistence

Pass a file path instead of `:memory:` and everything persists to disk.

In [ ]:
import tempfile, os

db_path = os.path.join(tempfile.mkdtemp(), "brain.db")

kg2 = KnowledgeGraph(db_path)
kg2.write("hello", "---\ntype: idea\n---\nPersisted to disk.")

# Reopen from the same file
kg3 = KnowledgeGraph(db_path)
print(f"Exists after reopen: {kg3.exists('hello')}")
print(f"Body: {kg3.body('hello')}")
print(f"Type: {kg3.frontmatter('hello')['type']}")

## 13. Interactive graph visualization

Generate an interactive graph — self-contained HTML with zero external deps.

Two views available via dropdown:
- **References** — `[[wikilink]]` edges, nodes sized by degree, colored by type
- **Tags** — edges between nodes sharing tags, opacity proportional to shared tag count

Drag nodes, pan, scroll-zoom, hover for tooltips.

In [ ]:
from IPython.display import HTML
from loopy.graph_viz import visualize

HTML(visualize(kg))